# Overview of Significance of Topic
- Batting speed in baseball
    - How does it contribute to the success of a team
    - Factors that contribute to/inhibit batting speed
    - How does the ability to predict/measure batting speed impact team/player performance predictions

# Formal Introduction to Topic
- Explain dataset
- Explain what we are trying to do/build – how a ML model could assist in predicting batting speed using this data set
- Pull from Project Proposal

# Variable Overview
- Break down dataset
- Include images to explain different variables
    - ex. hip rotation

# [ BUILDING THE MODEL/ EXPLAINING THE PROCESS]

# Conclusion of Results

# Bias
- Who benefits, who doesn't
- gender discrimination, accesibility
- pull from Project Proposal

# Application
- How could this be used by other teams/sports/groups 




# TO DO:
- David to write a quick paragraph on “Overview of Significance of Topic”
    - Julia will edit and bulk up with outside data
- David and Cece work on completing “Variable Overview” on Thursday
    - Julia to pull in images that explain different variables 
        - Ex. hip rotation visual 

